> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.11 - Data Model

## Exercícios

#### Q1. `dataclass`
Exercício adaptado de [codechalleng.es/bites/154/](https://codechalleng.es/bites/154/) e [codechalleng.es/bites/320/](https://codechalleng.es/bites/320/).

Neste desafio, você deve escrever uma `dataclass` chamada `Bite` que gerencia 3 atributos: `number`, `title` e `level`. Seus tipos são:
* `number` - `int`, 
* `title` - `str`, 
* `level` -  classe `Enum` chamada `BiteLevel` com os atributos `Beginner`, `Intermediate`, `Advanced`. 

Exemplo de dado: `{'number': 154, 'title': 'Escreva uma dataclass', 'level': BiteLevel.Intermediate}`

As características dessa classe são:
* O atributo`level` tem um valor padrão `BiteLevel.Beginner`
* Uma coleção de objetos `Bite` tem que ser ordenável somente pelo atributo `number`
* Implemente o método especial `__str__` para imprimir o Bite na forma `f'{number} - {title} ({level})'`

Teste sua classe executando o seguinte código:
```python
bites = []
bites.append(Bite(154, 'Escreva uma dataclass', 'Intermediate'))
bites.append(Bite(1, 'Some n valores'))
bites.append(Bite(37, 'Reescreva um loop com recursão', 'Intermediate'))

for b in bites.sort(): print(b)
# Ordem esperada na saída:
# 1 - Some n valores (Beginner)
# 37 - Reescreva um loop com recursão (Intermediate)
# 154 - Escreva uma dataclass (Intermediate)
```

In [1]:
from dataclasses import dataclass, field
from enum import Enum
from functools import total_ordering

class BiteLevel(Enum):
    Beginner = 'Beginner'
    Intermediate = 'Intermediate'
    Advanced = 'Advanced'

@total_ordering
@dataclass(order=True)
class Bite:
    number: int
    title: str
    level: BiteLevel = field(default=BiteLevel.Beginner, compare=False)

    def __str__(self):
        return f'{self.number} - {self.title} ({self.level.value})'

# Testando
if __name__ == "__main__":
    bites = []
    bites.append(Bite(154, 'Escreva uma dataclass', BiteLevel.Intermediate))
    bites.append(Bite(1, 'Some n valores'))
    bites.append(Bite(37, 'Reescreva um loop com recursão', BiteLevel.Intermediate))

    # Ordenação e impressão
    bites.sort()
    for b in bites:
        print(b)

1 - Some n valores (Beginner)
37 - Reescreva um loop com recursão (Intermediate)
154 - Escreva uma dataclass (Intermediate)


#### Q2. `Pydantic`
> Adaptada desse [tutorial de Pydantic](https://github.com/adonath/scipy-2023-pydantic-tutorial/tree/main) criado por [Axel Donath](https://github.com/adonath) e [Nick Langellier](https://github.com/nlangellier).

Observe a seguinte lista de observações da previsão do tempo em Murmansk, Russia.
```python
data_samples = [
    {
        "date": "2023-05-20",
        "temperature": 62.2,
        "isCelsius": False,
        "airQualityIndex": "24",
        "sunriseTime": "01:26",
        "sunsetTime": "00:00",
    },
    {
        "date": "2023-05-21",
        "temperature": "64.4",
        "isCelsius": "not true",
        "airQualityIndex": 23,
        "sunriseTime": "01:10",
        "sunsetTime": "00:16",
    },
    {
        "date": "2023-05-22",
        "temperature": 14.4,
        "airQualityIndex": 21,
    },
]
```

Escreva um script que calcule e imprima a temperatura média (em Celsius) em Murmansk para as datas fornecidas. Em seu script, você deve incluir um modelo Pydantic que registre com sucesso todos os elementos dados. Note que:

* Algumas amostras estão faltando dados. Você deve decidir quando o atributo pode ter um valor padrão ou quando definí-lo como opcional (`typing.Optional`). 
* Você precisará implementar pelo menos um validador de campo para transformar atributos. Dica: teste primeiro quais vão falhar :)



In [2]:
from pydantic import BaseModel, Field, validator
from typing import Optional
from datetime import datetime

class WeatherObservation(BaseModel):
    date: datetime
    temperature: float
    isCelsius: bool = Field(default=True)
    airQualityIndex: Optional[int] = None
    sunriseTime: Optional[str] = None
    sunsetTime: Optional[str] = None

    @validator("date", pre=True)
    def validate_date(cls, value):
        # Transforma a string em um objeto datetime
        return datetime.strptime(value, "%Y-%m-%d")

    @validator("temperature", pre=True)
    def validate_temperature(cls, value):
        # Garantir que o valor da temperatura seja float
        if isinstance(value, str):
            try:
                return float(value)
            except ValueError:
                raise ValueError(f"Temperatura inválida: {value}")
        return value

    @validator("isCelsius", pre=True)
    def validate_is_celsius(cls, value):
        # Converte valores não booleanos para booleanos
        if isinstance(value, str):
            return value.lower() == "true"
        return bool(value)

    @validator("airQualityIndex", pre=True)
    def validate_air_quality_index(cls, value):
        # Garante que o índice de qualidade do ar seja um inteiro
        if isinstance(value, str):
            try:
                return int(value)
            except ValueError:
                raise ValueError(f"Índice de qualidade do ar inválido: {value}")
        return value

    def get_temperature_in_celsius(self) -> float:
        # Converte a temperatura para Celsius, se necessário
        if not self.isCelsius:
            return (self.temperature - 32) * 5.0 / 9.0
        return self.temperature

# Lista de amostras de dados
data_samples = [
    {
        "date": "2023-05-20",
        "temperature": 62.2,
        "isCelsius": False,
        "airQualityIndex": "24",
        "sunriseTime": "01:26",
        "sunsetTime": "00:00",
    },
    {
        "date": "2023-05-21",
        "temperature": "64.4",
        "isCelsius": "not true",
        "airQualityIndex": 23,
        "sunriseTime": "01:10",
        "sunsetTime": "00:16",
    },
    {
        "date": "2023-05-22",
        "temperature": 14.4,
        "airQualityIndex": 21,
    },
]

# Processando os dados
observations = []
for sample in data_samples:
    try:
        observation = WeatherObservation(**sample)
        observations.append(observation)
    except Exception as e:
        print(f"Erro ao processar dados: {e}")

# Calculando a temperatura média em Celsius
if observations:
    avg_temp_celsius = sum(obs.get_temperature_in_celsius() for obs in observations) / len(observations)
    print(f"Temperatura média em Celsius: {avg_temp_celsius:.2f}")
else:
    print("Nenhuma observação válida para calcular a temperatura média.")

Temperatura média em Celsius: 16.39


/tmp/ipykernel_507237/1398158146.py:13: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("date", pre=True)
/tmp/ipykernel_507237/1398158146.py:18: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("temperature", pre=True)
/tmp/ipykernel_507237/1398158146.py:28: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators,

#### Q3
> Adaptada desse [tutorial de Pydantic](https://github.com/adonath/scipy-2023-pydantic-tutorial/tree/main) criado por [Axel Donath](https://github.com/adonath) e [Nick Langellier](https://github.com/nlangellier).

Na célula a seguir, coletamos dados reais de uma das principais APIs de previsão do tempo, [open-meteo](https://open-meteo.com/en/docs). Não se preocupe em entender esse código, o mais importante é entender o resultado que ele retorna, ilustrado a seguir para uma coleta da temperatura dos últimos 15 dias em Itabira -MG. Caso deseje alterar a cidade de coleta, basta alimentar a latitude e longitude desejada, como nas opções a seguir.
* Itabira: `'latitude': -19.656655787605846, 'longitude': -43.228922960534476`
* Bom Despacho: `'latitude': -19.726308457732443, 'longitude': -45.27462803349767`

```python
{
  "latitude": -19.5,
  "longitude": -43.375,
  "generationtime_ms": 0.01800060272216797,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 2.0,
  "hourly_units": {
    "time": "iso8601",
    "temperature_2m": "\u00b0C"
  },
  "hourly": {
    "time": [
      "2024-07-19T00:00",
      "2024-07-19T01:00",
      "2024-07-19T02:00",
      ...
    ],
    "temperature_2m": [
      21.9,
      20.9,
      20.0,
      ... 
    ]
  }
}
```

Você deve escrever um modelo Pydantic `OpenMeteo` que receba diretamente a resposta dessa API, através do comando:
```python
dados = OpenMeteo(**response)
``` 

Para comportar a estrutura hierárquica desse dicionário (é um dicionário com alguns dicionários internos), você deve criar uma classe Pydantic para cada dicionário interno (`HourlyUnits` e `Hourly`), com seus respectivos atributos. Essas classes serão atributos da classe principal `OpenMeteo`, que terá também os outros atributos da resposta (`latitude`, `longitude`, etc.).



In [3]:
import requests, json

url = 'https://api.open-meteo.com/v1/forecast'
lat, long = -19.656655787605846, -43.228922960534476
params = {'latitude': lat, 'longitude': long, 'elevation': 2,
          'hourly': 'temperature_2m', 'forecast_days': 15}
response = requests.get(url, params=params).json()
print(json.dumps(response, indent=2))

{
  "latitude": -19.5,
  "longitude": -43.375,
  "generationtime_ms": 0.5881786346435547,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 2.0,
  "hourly_units": {
    "time": "iso8601",
    "temperature_2m": "\u00b0C"
  },
  "hourly": {
    "time": [
      "2025-04-11T00:00",
      "2025-04-11T01:00",
      "2025-04-11T02:00",
      "2025-04-11T03:00",
      "2025-04-11T04:00",
      "2025-04-11T05:00",
      "2025-04-11T06:00",
      "2025-04-11T07:00",
      "2025-04-11T08:00",
      "2025-04-11T09:00",
      "2025-04-11T10:00",
      "2025-04-11T11:00",
      "2025-04-11T12:00",
      "2025-04-11T13:00",
      "2025-04-11T14:00",
      "2025-04-11T15:00",
      "2025-04-11T16:00",
      "2025-04-11T17:00",
      "2025-04-11T18:00",
      "2025-04-11T19:00",
      "2025-04-11T20:00",
      "2025-04-11T21:00",
      "2025-04-11T22:00",
      "2025-04-11T23:00",
      "2025-04-12T00:00",
      "2025-04-12T01:00",
      "2025-04-12T02:00"

In [4]:
from pydantic import BaseModel, Field
from typing import List
import requests
import json

class HourlyUnits(BaseModel):
    time: str = Field(..., description="Formato das unidades de tempo (ex: iso8601)")
    temperature_2m: str = Field(..., description="Unidade da temperatura (ex: °C)")

class Hourly(BaseModel):
    time: List[str] = Field(..., description="Lista de timestamps em formato ISO8601")
    temperature_2m: List[float] = Field(..., description="Lista de temperaturas em °C")

class OpenMeteo(BaseModel):
    latitude: float = Field(..., description="Latitude da localização")
    longitude: float = Field(..., description="Longitude da localização")
    generationtime_ms: float = Field(..., description="Tempo de geração dos dados pela API em milissegundos")
    utc_offset_seconds: int = Field(..., description="Deslocamento UTC em segundos")
    timezone: str = Field(..., description="Fuso horário da localização")
    timezone_abbreviation: str = Field(..., description="Abreviação do fuso horário")
    elevation: float = Field(..., description="Elevação em metros")
    hourly_units: HourlyUnits = Field(..., description="Unidades dos dados horários")
    hourly: Hourly = Field(..., description="Dados horários de temperatura e tempo")

# Código para coletar os dados da API e utilizar o modelo Pydantic
if __name__ == "__main__":
    url = 'https://api.open-meteo.com/v1/forecast'
    lat, long = -19.656655787605846, -43.228922960534476  # Coordenadas de Itabira-MG
    params = {
        'latitude': lat,
        'longitude': long,
        'elevation': 2,
        'hourly': 'temperature_2m',
        'forecast_days': 15
    }

    response = requests.get(url, params=params).json()
    print("Dados brutos da API:")
    print(json.dumps(response, indent=2))  # Para verificar os dados brutos retornados pela API

    # Criando o modelo OpenMeteo com os dados retornados
    try:
        dados = OpenMeteo(**response)
        print("\nDados validados pelo modelo Pydantic:")
        print(dados.json(indent=2))
    except Exception as e:
        print(f"Erro ao validar os dados da API: {e}")

Dados brutos da API:
{
  "latitude": -19.5,
  "longitude": -43.375,
  "generationtime_ms": 0.023245811462402344,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 2.0,
  "hourly_units": {
    "time": "iso8601",
    "temperature_2m": "\u00b0C"
  },
  "hourly": {
    "time": [
      "2025-04-11T00:00",
      "2025-04-11T01:00",
      "2025-04-11T02:00",
      "2025-04-11T03:00",
      "2025-04-11T04:00",
      "2025-04-11T05:00",
      "2025-04-11T06:00",
      "2025-04-11T07:00",
      "2025-04-11T08:00",
      "2025-04-11T09:00",
      "2025-04-11T10:00",
      "2025-04-11T11:00",
      "2025-04-11T12:00",
      "2025-04-11T13:00",
      "2025-04-11T14:00",
      "2025-04-11T15:00",
      "2025-04-11T16:00",
      "2025-04-11T17:00",
      "2025-04-11T18:00",
      "2025-04-11T19:00",
      "2025-04-11T20:00",
      "2025-04-11T21:00",
      "2025-04-11T22:00",
      "2025-04-11T23:00",
      "2025-04-12T00:00",
      "2025-04-12T01:00",
 

/tmp/ipykernel_507237/751111590.py:45: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(dados.json(indent=2))


#### Q4. 

Com os dados carregados na questão anterior plote um gráfico de linha, com a biblioteca de sua preferência, onde o eixo `x` são os timestamps (data e hora) e o eixo `y` é a temperatura medida.

In [1]:
import requests
import plotly.graph_objects as go
from datetime import datetime
from typing import List

# Função para buscar dados da API
def fetch_weather_data(lat: float, long: float, forecast_days: int = 15) -> dict:
    url = 'https://api.open-meteo.com/v1/forecast'
    params = {
        'latitude': lat,
        'longitude': long,
        'elevation': 2,
        'hourly': 'temperature_2m',
        'forecast_days': forecast_days
    }
    response = requests.get(url, params=params)
    return response.json()

# Função para processar os dados e gerar o gráfico
def plot_temperature(data: dict):
    # Extraindo os timestamps e as temperaturas
    timestamps: List[str] = data['hourly']['time']
    temperatures: List[float] = data['hourly']['temperature_2m']

    # Convertendo timestamps para objetos datetime
    dates = [datetime.fromisoformat(ts) for ts in timestamps]

    # Criando o gráfico interativo com Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates,
        y=temperatures,
        mode='lines+markers',
        name='Temperatura (°C)',
        line=dict(color='blue'),
        marker=dict(size=6)
    ))

    # Configurando o layout do gráfico
    fig.update_layout(
        title="Temperatura ao Longo do Tempo",
        xaxis_title="Data e Hora",
        yaxis_title="Temperatura (°C)",
        xaxis=dict(showgrid=True, tickformat="%d %b %H:%M"),
        yaxis=dict(showgrid=True),
        template="plotly_white",
        legend=dict(title="Legenda", x=0.8, y=1.2)
    )

    # Exibindo o gráfico
    fig.show()

if __name__ == "__main__":
    # Coordenadas de Itabira-MG
    lat, long = -19.656655787605846, -43.228922960534476

    # Buscando os dados da API
    weather_data = fetch_weather_data(lat, long)

    # Gerando o gráfico
    plot_temperature(weather_data)